In [4]:
import pandas as pd
import seaborn as sns 
import numpy as np
import glob
import matplotlib.pyplot as plt
%matplotlib inline
%config Completer.use_jedi = False

In [2]:
#Import FPKM, MAF and protein coding list files for required cancer type

#Add path to MAF and FPKM data colleted from TCGA
path = r'/Users/jake/OneDrive - University of Glasgow/Project/COAD/R_data'

#all csv files
all_files = glob.glob(path + "/*.csv")

#for loop through to read in tumour FPKM, normal FPKM and MAF files
for filename in all_files:
    
    if 'tumour' in filename:
        tumour_FPKM = pd.read_csv(filename)
    elif 'normal' in filename:
        normal_FPKM = pd.read_csv(filename)
    elif 'MAF' in filename:
        MAF = pd.read_csv(filename,low_memory=False,index_col=0)

        
#Load the protein coding gene list 
protein_coding_genes_file = '/Users/jake/OneDrive - University of Glasgow/Project/ensembl_GRCH38_protein_coding_genes.tsv'
protein_coding_genes = pd.read_csv(protein_coding_genes_file,sep='\t',index_col=0)

In [3]:
MAF.head()

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,Variant_Type,...,FILTER,CONTEXT,src_vcf_id,tumor_bam_uuid,normal_bam_uuid,case_id,GDC_FILTER,COSMIC,MC3_Overlap,GDC_Validation_Status
1,ATAD3B,83858,BCM,GRCh38,chr1,1485803,1485803,+,Nonsense_Mutation,SNP,...,PASS,TCAGTCGACCC,9130f121-b7ce-460f-b90c-8e31add6cd10,7de9d9e2-c4a4-4311-826f-973cb7987c66,861ad835-790a-47ad-8c03-06f7eb7b5710,7a70f061-9a6f-408e-a416-7f5295ceba3b,NaN,COSM1333470,True,Unknown
2,PLCH2,9651,BCM,GRCh38,chr1,2487195,2487195,+,Silent,SNP,...,PASS,AGGAGCCCTGC,9130f121-b7ce-460f-b90c-8e31add6cd10,7de9d9e2-c4a4-4311-826f-973cb7987c66,861ad835-790a-47ad-8c03-06f7eb7b5710,7a70f061-9a6f-408e-a416-7f5295ceba3b,NaN,COSM1340725;COSM1340726,True,Unknown
3,CHD5,26038,BCM,GRCh38,chr1,6146395,6146395,+,Missense_Mutation,SNP,...,PASS,AGTTGCGATAC,9130f121-b7ce-460f-b90c-8e31add6cd10,7de9d9e2-c4a4-4311-826f-973cb7987c66,861ad835-790a-47ad-8c03-06f7eb7b5710,7a70f061-9a6f-408e-a416-7f5295ceba3b,NaN,COSM911251,True,Unknown
4,IFNLR1,163702,BCM,GRCh38,chr1,24159060,24159060,+,Missense_Mutation,SNP,...,PASS,GGCCCGTGGCA,9130f121-b7ce-460f-b90c-8e31add6cd10,7de9d9e2-c4a4-4311-826f-973cb7987c66,861ad835-790a-47ad-8c03-06f7eb7b5710,7a70f061-9a6f-408e-a416-7f5295ceba3b,NaN,COSM1340840,True,Unknown
5,YTHDF2,51441,BCM,GRCh38,chr1,28769136,28769136,+,3'UTR,SNP,...,PASS,AAAAAAAGAAA,9130f121-b7ce-460f-b90c-8e31add6cd10,7de9d9e2-c4a4-4311-826f-973cb7987c66,861ad835-790a-47ad-8c03-06f7eb7b5710,7a70f061-9a6f-408e-a416-7f5295ceba3b,NaN,NaN,True,Unknown


In [26]:
#IMPACT col is the VEP status, with HIGH and MODERATE displaying variants that affect proteins
#Create new MAF DF with these values extracted

MAF_VEP_protein = MAF[(MAF['IMPACT'] == 'HIGH') | (MAF['IMPACT'] == 'MODERATE')]

In [27]:
#Write protein coding MAF to csv ... path.split('/')[5] = Cancer type

MAF_VEP_protein.to_csv(path.split('/')[5]+'_MAF_protein_coding_full.csv')

In [28]:
### FPKM DFs ####

In [29]:
#First rename gene ensembl col
tumour_FPKM.rename(columns={'Unnamed: 0':'Gene_ensembl_id'},inplace=True)
normal_FPKM.rename(columns={'Unnamed: 0':'Gene_ensembl_id'},inplace=True)

In [30]:
#Check DFs 
tumour_FPKM.head()

,Gene_ensembl_id,Gene_symbol,TCGA-AA-3867-01A-01R-1022-07,TCGA-CA-6719-01A-11R-1839-07,TCGA-NH-A50V-01A-11R-A28H-07,TCGA-AA-A01C-01A-01R-A00A-07,TCGA-AA-A02F-01A-01R-A089-07,TCGA-AA-A03F-01A-11R-A16W-07,TCGA-AA-A00O-01A-02R-A089-07,TCGA-AZ-4615-01A-01R-1410-07,...,TCGA-A6-6138-01A-11R-1774-07,TCGA-A6-2685-01A-01R-1410-07,TCGA-QG-A5YX-01A-11R-A28H-07,TCGA-AY-A54L-01A-11R-A28H-07,TCGA-A6-2671-01A-01R-1410-07,TCGA-NH-A8F8-01A-72R-A41B-07,TCGA-CM-5863-01A-21R-1839-07,TCGA-A6-3809-01A-01R-A278-07,TCGA-D5-6932-01A-11R-1928-07,TCGA-AA-3675-01A-02R-0905-07
0,ENSG00000000003,TSPAN6,60.706756,62.310848,28.054054,43.148277,49.288566,6.029851,15.781543,20.084270,...,43.579197,31.906153,47.325870,74.750409,22.444239,27.192429,33.208506,18.076069,29.713629,25.791329
1,ENSG00000000005,TNMD,0.065378,0.768211,0.087677,0.488598,0.058151,1.005012,0.000000,0.024084,...,0.699250,2.392969,0.602650,0.452964,0.265327,0.273898,5.345912,0.013534,0.199307,0.000000
2,ENSG00000000419,DPM1,53.021975,49.744823,39.211929,48.717118,47.276633,6.233657,59.815651,30.839994,...,54.079416,23.685697,39.968203,85.633379,43.145615,81.924110,20.560575,53.239013,52.921301,37.169671
3,ENSG00000000457,SCYL3,1.850407,1.996215,2.578204,1.157164,2.659188,1.140148,2.379475,1.706928,...,2.972966,1.609248,2.675526,1.867057,1.201758,2.562697,2.301442,2.823902,1.946377,4.566399
4,ENSG00000000460,C1orf112,1.234814,1.937095,0.980064,1.235927,4.581492,0.637249,0.782066,1.124192,...,2.282533,0.858440,1.899230,2.791183,2.154205,1.504133,0.872320,3.005428,1.630099,3.124514


In [31]:
#First filter for protein coding genes along gene_ensembl_id

tumour_FPKM_protein = tumour_FPKM[tumour_FPKM['Gene_ensembl_id'].isin(protein_coding_genes['ensembl_gene_id'])]
normal_FPKM_protein = normal_FPKM[normal_FPKM['Gene_ensembl_id'].isin(protein_coding_genes['ensembl_gene_id'])]

In [32]:
#check for duplicated genes - Each has two different Ensembl IDs 

tumour_FPKM_protein[tumour_FPKM_protein['Gene_symbol'].duplicated()]

,Gene_ensembl_id,Gene_symbol,TCGA-AA-3867-01A-01R-1022-07,TCGA-CA-6719-01A-11R-1839-07,TCGA-NH-A50V-01A-11R-A28H-07,TCGA-AA-A01C-01A-01R-A00A-07,TCGA-AA-A02F-01A-01R-A089-07,TCGA-AA-A03F-01A-11R-A16W-07,TCGA-AA-A00O-01A-02R-A089-07,TCGA-AZ-4615-01A-01R-1410-07,...,TCGA-A6-6138-01A-11R-1774-07,TCGA-A6-2685-01A-01R-1410-07,TCGA-QG-A5YX-01A-11R-A28H-07,TCGA-AY-A54L-01A-11R-A28H-07,TCGA-A6-2671-01A-01R-1410-07,TCGA-NH-A8F8-01A-72R-A41B-07,TCGA-CM-5863-01A-21R-1839-07,TCGA-A6-3809-01A-01R-A278-07,TCGA-D5-6932-01A-11R-1928-07,TCGA-AA-3675-01A-02R-0905-07
44529,ENSG00000258724,PINX1,0.030023,0.000000,0.011504,0.000000,0.000000,0.000000,0.000000,0.022119,...,0.012592,0.000000,0.000000,0.011244,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
49222,ENSG00000267281,ATF7,0.136954,0.167194,0.667191,0.511757,0.835293,0.661664,0.767311,0.259460,...,0.213358,0.553359,0.353480,0.681425,0.101056,1.128235,0.396141,0.145808,0.424963,0.206071
50079,ENSG00000269226,TMSB15B,0.017194,0.014693,0.026352,0.064247,0.015293,0.000000,0.000000,0.012667,...,0.000000,0.028605,0.012679,0.000000,0.012687,0.000000,0.013390,0.000000,0.006552,0.000000
56468,ENSG00000280987,MATR3,0.158523,0.274322,0.780508,0.192514,0.366594,0.158395,0.114881,0.140150,...,0.345743,0.105495,0.578653,0.460099,0.116971,1.050873,0.237641,0.817135,0.891009,0.606647


In [33]:
#For ATF7, ENSG00000170653 = AFT7, whereas ENSG00000267281 = ATF7-NPFF

display(tumour_FPKM_protein[tumour_FPKM_protein['Gene_symbol'] == 'ATF7'])


,Gene_ensembl_id,Gene_symbol,TCGA-AA-3867-01A-01R-1022-07,TCGA-CA-6719-01A-11R-1839-07,TCGA-NH-A50V-01A-11R-A28H-07,TCGA-AA-A01C-01A-01R-A00A-07,TCGA-AA-A02F-01A-01R-A089-07,TCGA-AA-A03F-01A-11R-A16W-07,TCGA-AA-A00O-01A-02R-A089-07,TCGA-AZ-4615-01A-01R-1410-07,...,TCGA-A6-6138-01A-11R-1774-07,TCGA-A6-2685-01A-01R-1410-07,TCGA-QG-A5YX-01A-11R-A28H-07,TCGA-AY-A54L-01A-11R-A28H-07,TCGA-A6-2671-01A-01R-1410-07,TCGA-NH-A8F8-01A-72R-A41B-07,TCGA-CM-5863-01A-21R-1839-07,TCGA-A6-3809-01A-01R-A278-07,TCGA-D5-6932-01A-11R-1928-07,TCGA-AA-3675-01A-02R-0905-07
12871,ENSG00000170653,ATF7,3.693301,5.472556,6.234170,3.906106,7.215266,2.382496,4.503972,3.736576,...,5.183876,3.313709,4.221508,4.201352,3.406515,5.763310,5.855046,2.179026,4.410650,3.952555
49222,ENSG00000267281,ATF7,0.136954,0.167194,0.667191,0.511757,0.835293,0.661664,0.767311,0.259460,...,0.213358,0.553359,0.353480,0.681425,0.101056,1.128235,0.396141,0.145808,0.424963,0.206071


In [34]:
# => rename Gene_symbol for ENSG00000267281 as ATF7-NPFF, based off of index location

index_val = tumour_FPKM_protein[tumour_FPKM_protein['Gene_ensembl_id'] == 'ENSG00000267281'].index

tumour_FPKM_protein.loc[index_val,'Gene_symbol'] = 'ATF7-NPFF'

#Display value to check its been renamed successfully
display(tumour_FPKM_protein[tumour_FPKM_protein['Gene_symbol'] == 'ATF7-NPFF'])

/Users/jake/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Gene_ensembl_id,Gene_symbol,TCGA-AA-3867-01A-01R-1022-07,TCGA-CA-6719-01A-11R-1839-07,TCGA-NH-A50V-01A-11R-A28H-07,TCGA-AA-A01C-01A-01R-A00A-07,TCGA-AA-A02F-01A-01R-A089-07,TCGA-AA-A03F-01A-11R-A16W-07,TCGA-AA-A00O-01A-02R-A089-07,TCGA-AZ-4615-01A-01R-1410-07,...,TCGA-A6-6138-01A-11R-1774-07,TCGA-A6-2685-01A-01R-1410-07,TCGA-QG-A5YX-01A-11R-A28H-07,TCGA-AY-A54L-01A-11R-A28H-07,TCGA-A6-2671-01A-01R-1410-07,TCGA-NH-A8F8-01A-72R-A41B-07,TCGA-CM-5863-01A-21R-1839-07,TCGA-A6-3809-01A-01R-A278-07,TCGA-D5-6932-01A-11R-1928-07,TCGA-AA-3675-01A-02R-0905-07
49222,ENSG00000267281,ATF7-NPFF,0.136954,0.167194,0.667191,0.511757,0.835293,0.661664,0.767311,0.25946,...,0.213358,0.553359,0.35348,0.681425,0.101056,1.128235,0.396141,0.145808,0.424963,0.206071


In [35]:
#Now set index to both Gene_symbol and Gene_ensembl_id, followed by transposing table

tumour_FPKM_protein_T = tumour_FPKM_protein.set_index(['Gene_symbol','Gene_ensembl_id']).transpose()
normal_FPKM_protein_T = normal_FPKM_protein.set_index(['Gene_symbol','Gene_ensembl_id']).transpose()

In [36]:
tumour_FPKM_protein_T

Gene_symbol,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,RPS4Y2,MATR3,AC005154.5,TMEM272,AC120114.4,BLACAT1,AC006978.2,AL365214.3,AL512506.3,GIMAP1-GIMAP5
Gene_ensembl_id,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,...,ENSG00000280969,ENSG00000280987,ENSG00000281039,ENSG00000281106,ENSG00000281348,ENSG00000281406,ENSG00000281593,ENSG00000281613,ENSG00000281883,ENSG00000281887
TCGA-AA-3867-01A-01R-1022-07,60.706756,0.065378,53.021975,1.850407,1.234814,1.545254,4.135409,34.276097,8.364260,8.990247,...,0.000000,0.158523,0.0,0.025358,0.000000,0.298832,0.079023,0.000000,0.000000,0.0
TCGA-CA-6719-01A-11R-1839-07,62.310848,0.768211,49.744823,1.996215,1.937095,3.081208,4.177262,43.155659,10.304510,13.205810,...,0.000000,0.274322,0.0,0.027087,0.485345,2.088031,0.033765,0.000000,0.025098,0.0
TCGA-NH-A50V-01A-11R-A28H-07,28.054054,0.087677,39.211929,2.578204,0.980064,1.828492,3.055178,23.782913,4.405802,10.217760,...,0.025462,0.780508,0.0,0.102019,0.145077,6.681474,0.090836,0.013346,0.157544,0.0
TCGA-AA-A01C-01A-01R-A00A-07,43.148277,0.488598,48.717118,1.157164,1.235927,0.962358,1.714293,38.444957,5.588669,6.992267,...,0.000000,0.192514,0.0,0.000000,0.000000,2.200446,0.000000,0.000000,0.000000,0.0
TCGA-AA-A02F-01A-01R-A089-07,49.288566,0.058151,47.276633,2.659188,4.581492,0.202121,1.333359,49.895112,7.937389,8.585949,...,0.059105,0.366594,0.0,0.000000,0.000000,3.783674,0.070287,0.000000,0.026122,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-NH-A8F8-01A-72R-A41B-07,27.192429,0.273898,81.924110,2.562697,1.504133,1.777106,1.821676,29.747612,7.910971,13.272776,...,0.000000,1.050873,0.0,0.068739,0.233271,8.577278,0.155794,0.000000,0.159228,0.0
TCGA-CM-5863-01A-21R-1839-07,33.208506,5.345912,20.560575,2.301442,0.872320,2.754771,4.498573,36.326234,6.133512,7.743226,...,0.000000,0.237641,0.0,0.059242,0.294858,1.690610,0.000000,0.000000,0.000000,0.0
TCGA-A6-3809-01A-01R-A278-07,18.076069,0.013534,53.239013,2.823902,3.005428,2.439958,5.861568,36.566763,7.917413,11.812828,...,0.000000,0.817135,0.0,0.041995,0.156764,0.080058,0.000000,0.000000,0.000000,0.0


In [37]:
#Drop the other three duplicated genes (only for time being, may add back in later)

duplicated_genes_drop = ['PINX1','TMSB15B','MATR3']

tumour_FPKM_protein_T.drop(duplicated_genes_drop,axis=1,inplace=True,level=0)
normal_FPKM_protein_T.drop(duplicated_genes_drop,axis=1,inplace=True,level=0)

In [38]:
#Add 'Cancer' column to DF, with value = 1 for tumour, value = 0 for normal

tumour_FPKM_protein_T['Cancer','Cancer'] = 1
normal_FPKM_protein_T['Cancer','Cancer'] = 0

In [39]:
tumour_FPKM_protein_T

Gene_symbol,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,RPS4Y2,AC005154.5,TMEM272,AC120114.4,BLACAT1,AC006978.2,AL365214.3,AL512506.3,GIMAP1-GIMAP5,Cancer
Gene_ensembl_id,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,...,ENSG00000280969,ENSG00000281039,ENSG00000281106,ENSG00000281348,ENSG00000281406,ENSG00000281593,ENSG00000281613,ENSG00000281883,ENSG00000281887,Cancer
TCGA-AA-3867-01A-01R-1022-07,60.706756,0.065378,53.021975,1.850407,1.234814,1.545254,4.135409,34.276097,8.364260,8.990247,...,0.000000,0.0,0.025358,0.000000,0.298832,0.079023,0.000000,0.000000,0.0,1
TCGA-CA-6719-01A-11R-1839-07,62.310848,0.768211,49.744823,1.996215,1.937095,3.081208,4.177262,43.155659,10.304510,13.205810,...,0.000000,0.0,0.027087,0.485345,2.088031,0.033765,0.000000,0.025098,0.0,1
TCGA-NH-A50V-01A-11R-A28H-07,28.054054,0.087677,39.211929,2.578204,0.980064,1.828492,3.055178,23.782913,4.405802,10.217760,...,0.025462,0.0,0.102019,0.145077,6.681474,0.090836,0.013346,0.157544,0.0,1
TCGA-AA-A01C-01A-01R-A00A-07,43.148277,0.488598,48.717118,1.157164,1.235927,0.962358,1.714293,38.444957,5.588669,6.992267,...,0.000000,0.0,0.000000,0.000000,2.200446,0.000000,0.000000,0.000000,0.0,1
TCGA-AA-A02F-01A-01R-A089-07,49.288566,0.058151,47.276633,2.659188,4.581492,0.202121,1.333359,49.895112,7.937389,8.585949,...,0.059105,0.0,0.000000,0.000000,3.783674,0.070287,0.000000,0.026122,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-NH-A8F8-01A-72R-A41B-07,27.192429,0.273898,81.924110,2.562697,1.504133,1.777106,1.821676,29.747612,7.910971,13.272776,...,0.000000,0.0,0.068739,0.233271,8.577278,0.155794,0.000000,0.159228,0.0,1
TCGA-CM-5863-01A-21R-1839-07,33.208506,5.345912,20.560575,2.301442,0.872320,2.754771,4.498573,36.326234,6.133512,7.743226,...,0.000000,0.0,0.059242,0.294858,1.690610,0.000000,0.000000,0.000000,0.0,1
TCGA-A6-3809-01A-01R-A278-07,18.076069,0.013534,53.239013,2.823902,3.005428,2.439958,5.861568,36.566763,7.917413,11.812828,...,0.000000,0.0,0.041995,0.156764,0.080058,0.000000,0.000000,0.000000,0.0,1


In [40]:
#Concat tumour FPKM and normmal FPKM

FPKM_protein = pd.concat([tumour_FPKM_protein_T,normal_FPKM_protein_T])

In [41]:
#Rename index to 'Patient_ID'
FPKM_protein.index.rename('Sample_ID',inplace=True)

In [42]:
FPKM_protein

Gene_symbol,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
Gene_ensembl_id,ENSG00000121410,ENSG00000148584,ENSG00000175899,ENSG00000166535,ENSG00000184389,ENSG00000128274,ENSG00000118017,ENSG00000094914,ENSG00000081760,ENSG00000114771,...,ENSG00000086827,ENSG00000174442,ENSG00000122952,ENSG00000198205,ENSG00000198455,ENSG00000070476,ENSG00000203995,ENSG00000162378,ENSG00000159840,ENSG00000074755
Sample_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-AA-3867-01A-01R-1022-07,0.052551,2.477200,57.378073,0.021620,0.000000,5.347004,0.000000,10.124001,3.888190,0.031552,...,8.337702,3.266961,11.951200,0.844530,2.250194,4.091899,0.011212,3.216321,87.952703,2.923495
TCGA-CA-6719-01A-11R-1839-07,0.005613,1.927244,74.944713,0.012317,0.000000,4.271353,0.025395,11.569886,1.872340,1.159435,...,10.875540,6.107697,11.915234,1.711313,4.772996,4.795855,0.009581,5.434010,72.500361,5.158080
TCGA-NH-A50V-01A-11R-A28H-07,0.035237,0.550182,51.276491,0.024852,0.000000,5.097944,0.056933,10.462945,3.612377,0.193436,...,7.953033,1.813204,14.288954,1.255144,4.156986,5.492997,0.038664,3.399480,57.827137,3.430372
TCGA-AA-A01C-01A-01R-A00A-07,0.049092,1.914795,45.653386,0.013464,0.000000,5.011742,0.000000,15.683433,3.708620,0.117902,...,7.460225,3.650127,22.783866,1.123639,2.886182,3.341388,0.000000,2.662764,116.008374,5.140081
TCGA-AA-A02F-01A-01R-A089-07,0.011685,1.145542,28.787736,0.006410,0.000000,2.099586,0.000000,15.396349,2.902797,0.028064,...,7.259496,11.352859,21.240253,1.650287,4.630294,6.636417,0.000000,5.230433,68.590155,2.939338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-A6-2682-11A-01R-A32Z-07,0.044713,7.063770,105.433200,0.007358,0.000000,2.197177,0.000000,9.090274,3.171970,0.139603,...,6.268062,2.474975,12.888024,0.849228,2.239781,4.163356,0.049608,4.066180,48.875219,15.281410
TCGA-AA-3522-11A-01R-A32Z-07,0.007344,8.778757,103.172026,0.006043,0.028758,2.124263,0.000000,8.528042,2.862644,0.026457,...,6.946885,3.200441,15.142902,1.055050,2.443330,4.262810,0.009401,4.717286,28.596000,17.377093


In [43]:
#Save the full FPKM to csv
FPKM_protein.to_csv(path.split('/')[5]+'_FPKM_protein_all.csv',sep='\t')

In [44]:
#Write normal and tumour FPKM files to csv

tumour_FPKM_protein_T.to_csv(path.split('/')[5]+'_FPKM_tumour.csv',sep='\t')
normal_FPKM_protein_T.to_csv(path.split('/')[5]+'_FPKM_normal.csv',sep='\t')